# Задача о классификации таблеток

## Этап 1
Загрузите датасеты для обучения и тестирования, для каждого определите этапы предобработки. Полученные датасеты, упакованные для использования в обучении, поместите в переменные train_loader и val_loader.
Выведите количество классов, количество изображений в обоих датасетах.
Результаты этапа
Готовые для обучения датасеты с предобработкой, загруженные в DataLoader.
Количество классов, изображений в обучающем и валидационном датасетах.

In [1]:
import os
import sys

# Добавляем папку src в путь поиска, чтобы импортировать модули
sys.path.append('./src')

from data_utils import get_data_loaders

# Настройки
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'
BATCH_SIZE = 32

# Загрузка данных
train_loader, val_loader, train_ds, val_ds = get_data_loaders(TRAIN_DIR, TEST_DIR, batch_size=BATCH_SIZE)

# Вывод статистики
num_classes = len(train_ds.classes)
num_train = len(train_ds)
num_val = len(val_ds)

print(f"--- Статистика датасета ---")
print(f"Количество классов: {num_classes}")
print(f"Изображений для обучения: {num_train}")
print(f"Изображений для валидации: {num_val}")
print(f"Названия классов: {train_ds.classes}")

--- Статистика датасета ---
Количество классов: 11
Изображений для обучения: 308
Изображений для валидации: 66
Названия классов: ['acc_long_600_mg', 'advil_ultra_forte', 'akineton_2_mg', 'algoflex_forte_dolo_400_mg', 'algoflex_rapid_400_mg', 'algopyrin_500_mg', 'ambroxol_egis_30_mg', 'apranax_550_mg', 'aspirin_ultra_500_mg', 'atoris_20_mg', 'atorvastatin_teva_20_mg']


## Этап 2. Объявление модели
Создайте классификатор изображений, подходящий для классификации таблеток. Если часть весов нужно заморозить, сделайте это. Итоговый классификатор, готовый к обучению или дообучению, поместите в переменную model.
Результат этапа — классификатор, готовый к обучению, в переменной model.

In [2]:
import torch
import sys
import os

# Добавляем путь к папке src, если еще не добавлен
sys.path.append('./src')

# Импортируем нашу функцию из созданного файла
from model import get_pill_classifier

# Предполагаем, что num_classes определен на Этапе 1
# num_classes = len(train_loader.dataset.classes)

# Инициализируем модель
model = get_pill_classifier(num_classes=num_classes, freeze_backbone=True)

# Переносим на устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"--- Результат Этапа 2 ---")
print(f"Модель готова к обучению в переменной 'model'")
print(f"Используемое устройство: {device}")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\nkote/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


--- Результат Этапа 2 ---
Модель готова к обучению в переменной 'model'
Используемое устройство: cpu


## Этап 3. Обучение или дообучение

Обучите или дообучите классификатор на датасете таблеток. Сохраните обученную модель в файл meds_classifier.pt.  
Обучение может занять какое-то время. Чтобы модель обучалась быстрее, используйте GPU. 

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import sys
import os

# Подключаем наши модули
sys.path.append('/src')
from train_utils import train_one_epoch, validate

# 1. Настройка устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Параметры обучения
criterion = nn.CrossEntropyLoss()
# Обучаем только размороженные параметры (голову модели)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5 # Можно изменить в зависимости от прогресса метрики

print(f"Начинаем обучение на устройстве: {device}")

for epoch in range(num_epochs):
    # Обучение
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    
    # Валидация
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    print(f"Эпоха [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")
    print("-" * 30)

# 3. Сохранение модели
# Перед сохранением и расчетом финальных метрик переводим модель на CPU
model.to('cpu')
torch.save(model.state_dict(), 'models/meds_classifier.pt')
print("Модель сохранена в файл models/meds_classifier.pt")

c:\Users\nkote\sprint3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Начинаем обучение на устройстве: cpu


Validation: 100%|██████████| 3/3 [00:27<00:00,  9.30s/it]


Эпоха [1/5]
Train Loss: 2.4142 | Train Acc: 0.1169
Val Loss: 2.2360 | Val Acc: 0.2879
------------------------------


Validation: 100%|██████████| 3/3 [00:30<00:00, 10.17s/it]


...

Эпоха [3/5]
Train Loss: 2.0219 | Train Acc: 0.3442
Val Loss: 1.6258 | Val Acc: 0.6818
------------------------------


Validation: 100%|██████████| 3/3 [00:25<00:00,  8.38s/it]


Эпоха [4/5]
Train Loss: 1.8608 | Train Acc: 0.4286
Val Loss: 1.4141 | Val Acc: 0.7727
------------------------------


Validation: 100%|██████████| 3/3 [00:30<00:00, 10.29s/it]


Эпоха [5/5]
Train Loss: 1.6877 | Train Acc: 0.4708
Val Loss: 1.3390 | Val Acc: 0.6970
------------------------------
Модель сохранена в файл models/meds_classifier.pt


# Этап 4. Оценка качества
Рассчитайте метрики обученного классификатора, используя проверочный датасет. 


Результаты этапа

Распечатка Precision, Recall, F1 для каждого класса + общая accuracy для классификатора таблеток с accuracy выше 75%.

Текст с ответами на вопросы:

На каких 5 классах модель ошибается чаще всего?

Почему модель может ошибаться на этих классах?

На каких классах модель не совершает ошибок?

Почему эти классы модель распознаёт безошибочно?

Как можно улучшить точность классификатора?

Как ещё можно проанализировать результаты и ошибки модели?

In [5]:
import pandas as pd
import sys
import os

# Подключаем модуль
sys.path.append('/src')
from eval import evaluate_model, get_metrics_report

# 1. Получаем предсказания
y_true, y_pred = evaluate_model(model, val_loader, device)

# 2. Считаем метрики
class_names = val_loader.dataset.classes
report = get_metrics_report(y_true, y_pred, class_names)

# 3. Красивый вывод через Pandas
df_report = pd.DataFrame(report).transpose()

# Выводим точность
total_acc = df_report.loc['accuracy', 'precision'] # В dict-версии accuracy лежит здесь
print(f"Общая точность классификатора: {total_acc:.2%}")
print("\nМетрики по каждому классу:")
# Показываем только классы (убираем строки average и accuracy в конце)
display(df_report.iloc[:-3, :]) 

# 4. Подготовка данных для анализа ошибок
# Добавим колонку с названием класса для удобства сортировки
df_classes = df_report.iloc[:-3, :].copy()
df_classes['class_name'] = df_classes.index

Общая точность классификатора: 69.70%

Метрики по каждому классу:


c:\Users\nkote\sprint3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nkote\sprint3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nkote\sprint3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,precision,recall,f1-score,support
acc_long_600_mg,1.000000,1.000000,1.000000,6.0
advil_ultra_forte,0.833333,0.833333,0.833333,6.0
akineton_2_mg,0.444444,0.666667,0.533333,6.0
algoflex_forte_dolo_400_mg,1.000000,0.833333,0.909091,6.0
algoflex_rapid_400_mg,1.000000,0.833333,0.909091,6.0
algopyrin_500_mg,0.352941,1.000000,0.521739,6.0
ambroxol_egis_30_mg,1.000000,0.500000,0.666667,6.0
apranax_550_mg,1.000000,0.666667,0.800000,6.0
aspirin_ultra_500_mg,0.000000,0.000000,0.000000,6.0
atoris_20_mg,0.750000,0.500000,0.600000,6.0


In [8]:
# Автоматическая генерация анализа ошибок
df_classes = df_report.iloc[:-3, :].copy()
df_classes['f1-score'] = df_classes['f1-score'].astype(float)

print(df_classes.sort_values(by='f1-score'))
# 1. Находим худшие и лучшие классы
worst_5 = df_classes.sort_values(by='f1-score').head(5).index.tolist()
perfect_classes = df_classes[df_classes['f1-score'] > 0.99].index.tolist()

# Формируем текст
analysis_text = f"""
### Анализ результатов модели

1. На каких 5 классах модель ошибается чаще всего?
Модель испытывает наибольшие трудности со следующими классами: {', '.join(worst_5)}.

2. Почему модель может ошибаться на этих классах?
Основные причины ошибок в этой группе:
- Визуальное сходство: Эти препараты, скорее всего, имеют идентичную форму (круглые) и цвет (белые). 
- Низкая детализация: Отличительные знаки (насечки, гравировка) могут быть плохо видны при текущем разрешении изображения.
- Дисбаланс: Возможно, в обучающей выборке для этих классов было меньше примеров или они были менее разнообразными.

3. На каких классах модель не совершает ошибок?
Идеальную или почти идеальную точность модель показала на классах: {', '.join(perfect_classes) if perfect_classes else 'классы с F1 > 0.99 не обнаружены'}.
4. Почему эти классы модель распознаёт безошибочно?
- Уникальность: Эти таблетки имеют специфическую форму (капсулы, треугольники) или яркий цвет, который не встречается у других препаратов.
- Высокий контраст: Четкие границы объекта и фона облегчают работу сверточных слоев нейросети.

5. Как можно улучшить точность классификатора?
- Fine-tuning: Разморозить последние слои ResNet и дообучить их с низким Learning Rate.
- Увеличение разрешения: Использовать входной размер 448x448 вместо 224x224, чтобы модель видела текст на таблетках.
- Аугментация: Добавить случайные изменения яркости и резкости, имитируя разные камеры смартфонов.

6. Как ещё можно проанализировать результаты и ошибки модели?
- Confusion Matrix:  Построить тепловую карту, чтобы увидеть, какие именно пары классов "слипаются" между собой.
- Визуализация ошибок: Вывести изображения, на которых модель дала неверный ответ с максимальной уверенностью (высоким Softmax), чтобы понять логику ложных срабатываний.
"""

print(analysis_text)

                            precision    recall  f1-score  support
aspirin_ultra_500_mg         0.000000  0.000000  0.000000      6.0
algopyrin_500_mg             0.352941  1.000000  0.521739      6.0
akineton_2_mg                0.444444  0.666667  0.533333      6.0
atoris_20_mg                 0.750000  0.500000  0.600000      6.0
ambroxol_egis_30_mg          1.000000  0.500000  0.666667      6.0
atorvastatin_teva_20_mg      0.714286  0.833333  0.769231      6.0
apranax_550_mg               1.000000  0.666667  0.800000      6.0
advil_ultra_forte            0.833333  0.833333  0.833333      6.0
algoflex_forte_dolo_400_mg   1.000000  0.833333  0.909091      6.0
algoflex_rapid_400_mg        1.000000  0.833333  0.909091      6.0
acc_long_600_mg              1.000000  1.000000  1.000000      6.0

### Анализ результатов модели

1. На каких 5 классах модель ошибается чаще всего?
Модель испытывает наибольшие трудности со следующими классами: aspirin_ultra_500_mg, algopyrin_500_mg, akineton_2